### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
### OpenAi API KEY and Open Source models --- Llama, Gemma, Mistral -- Groq

import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [12]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.1-8b-instant")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x124512960>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x125372090>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [13]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

response = model.invoke(messages)
response

AIMessage(content='Bonjour Comment allez-vous ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 47, 'total_tokens': 54, 'completion_time': 0.012193415, 'completion_tokens_details': None, 'prompt_time': 0.002489333, 'prompt_tokens_details': None, 'queue_time': 0.005473396, 'total_time': 0.014682748}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_29e590f0c0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--d4b18cf0-2dea-4832-ba95-a6736c5120ec-0', usage_metadata={'input_tokens': 47, 'output_tokens': 7, 'total_tokens': 54})

In [14]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(response)

'Bonjour Comment allez-vous ?'

In [15]:
### Using LCEL - chain the components
chain = model | parser
chain.invoke(messages)

'Bonjour Comment allez-vous ?'

In [16]:
### Prompot Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template),
        ("user", "{text}")
    ]
)

In [17]:
result = prompt.invoke({"language": "spanish", "text": "hello"})

In [18]:
result.to_messages()

[SystemMessage(content='Translate the following into spanish:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello', additional_kwargs={}, response_metadata={})]

In [19]:
### Chaining together components with LCEL
chain = prompt | model | parser
chain.invoke({"language": "spanish", "text": "Good morning"})

'Buenos días.'